# Análise Exploratória de Dados usando Pandas

EDA é o processo de obter, preparar e apresentar dados com o objetivo de obter insights que direcionem o negócio de maneira eficiente e eficaz.

<img src="assets/data_prep_pipeline.png" />

É o processo mais demorado, visto que exige atenção especial para preparar dados corretamente a fim de não produzir resultados enviesados ou mesmos incorretos. 

Vamos começar a analisar os dados mais básicos de um dataset simples. O objetivo aqui é se familiarizar com o pandas, criar funções e sedimentar o conteúdo já visto, preparando o terreno para os próximos assuntos que serão vistos. 

Documentação oficial do Pandas para consulta: [link](https://pandas.pydata.org/docs/user_guide/index.html#user-guide) 

## Pandas in a nutshell

In [6]:
import pandas as pd
lemonades = pd.read_csv('bases/Lemonades.csv', sep = ';')

lemonades.head(10) # mostra as n primeiras linhas do dataset - 5 é o padrão

,Date,Location,Lemon,Orange,Temperature,Leaflets,Price
0,07/01/2016,Park,97,67,70,90.0,0.25
1,07/02/2016,Park,98,67,72,90.0,0.25
2,07/03/2016,Park,110,77,71,104.0,0.25
3,07/04/2016,Beach,134,99,76,98.0,0.25
4,07/05/2016,Beach,159,118,78,135.0,0.25
5,07/06/2016,Beach,103,69,82,90.0,0.25
6,07/06/2016,Beach,103,69,82,90.0,0.25
7,07/07/2016,Beach,143,101,81,135.0,0.25
8,NaN,Beach,123,86,82,113.0,0.25
9,07/09/2016,Beach,134,95,80,126.0,0.25


In [2]:
lemonades.columns

Index(['Date', 'Location', 'Lemon', 'Orange', 'Temperature', 'Leaflets',
       'Price'],
      dtype='object')

É sempre importante olhar os dados e fazer algumas indagações iniciais:
> Quais os tipos de dados presentes no dataset? 

> Existem valores faltantes

> Quantas colunas e quantas linhas estão presentes no dataset

> Das colunas com valores numéricos, que estatísticas eu consigo obter? Soma, média, desvio padrão, etc..

Vamos responder cada uma delas:

In [3]:
# os tipos de dados de cada coluna podem ser obtidos usando o método info()
lemonades.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32 entries, 0 to 31
Data columns (total 7 columns):
Date           31 non-null object
Location       32 non-null object
Lemon          32 non-null int64
Orange         32 non-null int64
Temperature    32 non-null int64
Leaflets       31 non-null float64
Price          32 non-null float64
dtypes: float64(2), int64(3), object(2)
memory usage: 1.9+ KB


Com esse método, foi possível saber se há ou não valores faltantes no dataset. Observando-o, vemos que faltam dados nas colunas Date e Leaflets. 

Para eximir qualquer duúvida a respeito, podemos usar outro método:

In [4]:
# retorna a soma dos valores faltantes em cada coluna do dataset. 
lemonades.isna().sum()

Date           1
Location       0
Lemon          0
Orange         0
Temperature    0
Leaflets       1
Price          0
dtype: int64

A quantidade de linhas e colunas foi também mostrada ao usarmos o método info(). Entretanto, há maneiras mais simples de obter essa informação. 

Qual atributo poderíamos usar aqui que retornaria a quantidade de linhas e colunas?

In [5]:
#Resposta
lemonades.shape

(32, 7)

In [6]:
#localização
#lemonades.loc[:,'Lemon'] # usando strings - todas as linhas da coluna Date
lemonades.loc[1:6,'Date':'Orange'] # selecionando linhas e colunas específicas

,Date,Location,Lemon,Orange
1,07/02/2016,Park,98,67
2,07/03/2016,Park,110,77
3,07/04/2016,Beach,134,99
4,07/05/2016,Beach,159,118
5,07/06/2016,Beach,103,69
6,07/06/2016,Beach,103,69


Podemos escrever várias funções para obtermos as estatísticas básicas das colunas numéricas do nosso dataset. 

Exemplo:

In [7]:
def media(coluna):
    return lemonades[coluna].sum()/len(lemonades)
    
media('Orange')

80.0

Entretanto, essa maneira não seria a mais apropriada (Pythonica). Qual método podemos usar para retornar as principais estatísticas de nosso dataset sem precisar escrever funções?

In [8]:
#resposta
lemonades.describe()

,Lemon,Orange,Temperature,Leaflets,Price
count,32.000000,32.000000,32.000000,31.000000,32.000000
mean,116.156250,80.000000,78.968750,108.548387,0.354687
std,25.823357,21.863211,4.067847,20.117718,0.113137
min,71.000000,42.000000,70.000000,68.000000,0.250000
25%,98.000000,66.750000,77.000000,90.000000,0.250000
50%,113.500000,76.500000,80.500000,108.000000,0.350000
75%,131.750000,95.000000,82.000000,124.000000,0.500000
max,176.000000,129.000000,84.000000,158.000000,0.500000


#### Dados categóricos

Até agora, tratamos, principalmente, dados numéricos, porém, há em nosso dataset dados categóricos. Como podemos analisar o comportamento desses dados?

In [9]:
lemonades['Location'].value_counts()

Beach    17
Park     15
Name: Location, dtype: int64

In [10]:
lemonades.describe(include='O')

,Date,Location
count,31,32
unique,30,2
top,07/06/2016,Beach
freq,2,17


## Tratamento do Dataset

Anteriormente, descobrimos que há elementos faltantes em nosso dataset. Algumas implicações podem ocorrer se treinarmos um modelo de machine learning em dados com essas inconsistências:
> criar um viés nos dados

> erro ao treinar um modelo de machine learning

Além desse problema, há outro que devemos levar em consideração:linhas duplicadas. Vamos, passo a passo, resolver esses problemas.

### Lidando com dados faltantes

In [11]:
#verificando quais colunas possuem dados faltantes
lemonades.isna().sum()

Date           1
Location       0
Lemon          0
Orange         0
Temperature    0
Leaflets       1
Price          0
dtype: int64

Quando falamos de dados numéricos, uma maneira simples de tratar os dados faltantes é obter a média (ou mediana) dos valores presentes nessa coluna e preencher os dados faltantes com esse valor. 

Uma solução seria:

In [12]:
lemonades['Leaflets'].median()

108.0

In [13]:
lemonades['Leaflets'].fillna((lemonades['Leaflets'].median()), inplace=True)
#lemonades['Leaflets'] = lemonades['Leaflets'].fillna(lemonades['Leaflets'].median())

In [14]:
# verificando se o problema foi resolvido
lemonades.isna().sum()

Date           1
Location       0
Lemon          0
Orange         0
Temperature    0
Leaflets       0
Price          0
dtype: int64

Quando falamos, entretando, de data, o problema pode ser um pouco mais difícil de ser resolvido caso o formato não seja datetime. Se não for, precisamos antes converter para esse tipo e depois tratar os dados faltantes.

In [15]:
# Verificando o tipo de dado presente na coluna Date
lemonades['Date'].dtypes

dtype('O')

In [ ]:
lemonades

In [7]:
from datetime import datetime
#Convertendo para o formato correto
lemonades['Date'] = pd.to_datetime(lemonades['Date'], format="%m/%d/%Y")     

# converte cada valor para segundos
tmp = lemonades['Date'].apply(lambda t: (t-datetime(1970,1,1)).total_seconds())
# faz a interpolação do valor faltante
tmp.interpolate(inplace=True)    
# converte de volta para data
lemonades['Date'] = pd.to_datetime(tmp, unit='s') 
lemonades['Date'] = lemonades['Date'].apply(lambda t: t.date())
#imprimindo para verificar
lemonades.head(3)

,Date,Location,Lemon,Orange,Temperature,Leaflets,Price
0,2016-07-01,Park,97,67,70,90.0,0.25
1,2016-07-02,Park,98,67,72,90.0,0.25
2,2016-07-03,Park,110,77,71,104.0,0.25


In [17]:
lemonades['Date'] = pd.to_datetime(lemonades['Date'], format="%m/%d/%Y") 
lemonades.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32 entries, 0 to 31
Data columns (total 7 columns):
Date           31 non-null datetime64[ns]
Location       32 non-null object
Lemon          32 non-null int64
Orange         32 non-null int64
Temperature    32 non-null int64
Leaflets       32 non-null float64
Price          32 non-null float64
dtypes: datetime64[ns](1), float64(2), int64(3), object(1)
memory usage: 1.9+ KB


In [23]:
from datetime import datetime
tmp = lemonades['Date'].apply(lambda t: (t-datetime(1970,1,1)).total_seconds())

86400.0

In [24]:
tmp.interpolate(inplace=True)  

In [26]:
lemonades['Date'] = pd.to_datetime(tmp, unit='s') 
lemonades.head(2)

,Date,Location,Lemon,Orange,Temperature,Leaflets,Price
0,2016-07-01,Park,97,67,70,90.0,0.25
1,2016-07-02,Park,98,67,72,90.0,0.25


In [29]:
#lemonades['Date'] = lemonades['Date'].apply(lambda t: t.date())
lemonades.head()

,Date,Location,Lemon,Orange,Temperature,Leaflets,Price
0,2016-07-01,Park,97,67,70,90.0,0.25
1,2016-07-02,Park,98,67,72,90.0,0.25
2,2016-07-03,Park,110,77,71,104.0,0.25
3,2016-07-04,Beach,134,99,76,98.0,0.25
4,2016-07-05,Beach,159,118,78,135.0,0.25


Agora que tratamos todos os valores faltantes, precisamos lidar com os dados duplicados. O Pandas lida com isso de forma simples e intuitiva

In [8]:
# verificando se há dados duplicados
lemonades[lemonades.duplicated()]

,Date,Location,Lemon,Orange,Temperature,Leaflets,Price
6,2016-07-06,Beach,103,69,82,90.0,0.25


In [9]:
# eliminando os valores duplicados
lemonades = lemonades.drop_duplicates()

In [10]:
# verificando se há dados duplicados
lemonades[lemonades.duplicated()]

,Date,Location,Lemon,Orange,Temperature,Leaflets,Price


In [33]:
# para salvar a base corrigida
lemonades.to_csv('resultado.csv', sep = ';')

## Treinando a criação de funções

Vamos criar algumas funções para retornar informações úteis para nossa análise. Antes, vamos preparar o dataset

### 1) Adicione uma coluna chamada "Sales" que contém o total de vendas de limão e laranja

In [11]:
# Resposta
lemonades["Sales"] = lemonades["Lemon"] + lemonades["Orange"]
lemonades.head()

,Date,Location,Lemon,Orange,Temperature,Leaflets,Price,Sales
0,2016-07-01,Park,97,67,70,90.0,0.25,164
1,2016-07-02,Park,98,67,72,90.0,0.25,165
2,2016-07-03,Park,110,77,71,104.0,0.25,187
3,2016-07-04,Beach,134,99,76,98.0,0.25,233
4,2016-07-05,Beach,159,118,78,135.0,0.25,277


### 2) Adicione uma coluna chamada "Revenue" que calcula o lucro (venda*preço)

In [12]:
# Resposta
lemonades["Revenue"] = lemonades["Sales"] * lemonades["Price"]
lemonades.head()

,Date,Location,Lemon,Orange,Temperature,Leaflets,Price,Sales,Revenue
0,2016-07-01,Park,97,67,70,90.0,0.25,164,41.00
1,2016-07-02,Park,98,67,72,90.0,0.25,165,41.25
2,2016-07-03,Park,110,77,71,104.0,0.25,187,46.75
3,2016-07-04,Beach,134,99,76,98.0,0.25,233,58.25
4,2016-07-05,Beach,159,118,78,135.0,0.25,277,69.25


### 3) Escreva uma função que retorne o lucro total

In [13]:
# Resposta
def total_revenue(df):
    return df["Revenue"].sum()
    
total_revenue(lemonades)

2138.0

### 4) Escreva uma função que receba dois parâmetros, dataset e temp(int). Se temp for 1, ele retorna a máxima temperatura observada no conjunto de dados; se 0, retorna a média do período observado; se -1, retorna a temperatura mínima.

In [14]:
# Resposta
def funcao_temp(dataset, temp):
    if(temp == 1):
        return dataset["Temperature"].max()
    elif(temp == 0):
        return dataset["Temperature"].mean()
    elif(temp == -1):
        return dataset["Temperature"].min()
    
print(funcao_temp(lemonades, -1))
print(funcao_temp(lemonades, 0))
print(funcao_temp(lemonades, 1))

70
78.87096774193549
84


### 5) Escreva uma função que receba dois parâmetros (dataset, localização) e retorne o dataset com o preço do limão e laranja ajustados em 15% se a localização for 'Park' ou ajustados em 10% se a localização for 'Beach'.

In [17]:
# Resposta
def get_adjusted_price(dataset, location):
    multiplier = 1.10 if location == "Beach" else 1.15 if location == "Park" else 1
    return dataset[["Lemon", "Orange"]] * multiplier
        
print(get_adjusted_price(lemonades, "Beach").head())
print(get_adjusted_price(lemonades, "Park").head())

   Lemon  Orange
0  106.7    73.7
1  107.8    73.7
2  121.0    84.7
3  147.4   108.9
4  174.9   129.8
    Lemon  Orange
0  111.55   77.05
1  112.70   77.05
2  126.50   88.55
3  154.10  113.85
4  182.85  135.70
   Lemon  Orange
0     97      67
1     98      67
2    110      77
3    134      99
4    159     118


In [15]:
lemonades[['Lemon','Orange']]

,Lemon,Orange
0,97,67
1,98,67
2,110,77
3,134,99
4,159,118
5,103,69
7,143,101
8,123,86
9,134,95
10,140,98
